In [3]:
import pdfplumber
import re
import pandas as pd
import sys

In [4]:
# Move to the folder of submissions
%cd C:\Users\ascadmin\Desktop\PotentialCustomersSolution\Automation-of-Loss-Runs\submissions


C:\Users\ascadmin\Desktop\PotentialCustomersSolution\Automation-of-Loss-Runs\submissions


In [5]:
!pwd

/c/Users/ascadmin/Desktop/PotentialCustomersSolution/Automation-of-Loss-Runs/submissions


In [8]:
def PDFtoText(path_to_pdf: str):
    '''
    converts PDF to a string variable

    input: string
    output: string variable containing losses with labels from PDF
    '''
    # initiate text variable

    text = " "

    with pdfplumber.open(path_to_pdf) as pdf:
        pages = pdf.pages
        for page in pages:
            new = page.extract_text()
            print(new)
            text += new     

    # Find the index positions of 'LOSSES' and 'ADDITIONAL INFORMATION'
    losses_index = text.find('LOSSES')
    additional_info_index = text.find('ADDITIONAL INFORMATION')

    # Find the index position of 'LARGE LOSSES'
    large_losses_index = text.find('LARGE LOSSES')

    # Slice the string to get the portions after 'LOSSES' and 'ADDITIONAL INFORMATION'
    losses = text[losses_index + len('LOSSES'):large_losses_index] if losses_index != -1 else ''

    # Slice the string to get the parts after 'LARGE LOSSES'
    large_losses = text[large_losses_index + len('LARGE LOSSES'):additional_info_index] if large_losses_index != -1 else ''


    print("LOSSES:\n", losses)
    print("\nLARGE LOSSES:\n", large_losses)  

    return losses



In [10]:
fernlea = PDFtoText("Fernlea.pdf")

A/C: Fernlea Industries, Inc. UMBRELLA LIABILITY
R/I: Upland Capital Group June 4, 2024
From: Brian Kohout Telephone: (630) 209-7195
GC RENEWAL: No
CLIENT RENEWAL: Yes
POLICY PERIOD: June 1, 2024 To June 1, 2025
REINSURED PERIOD: June 1, 2024 To June 1, 2025
TYPE OF INSURANCE: Excess Liability
COMPANY POLICY LIMIT(S): $5,000,000 Each Occurrence / $5,000,000 Aggregate Excess of
- Any/All Underlying Insurance and/or Self-Insured Retention
- (SIR)
TOTAL POLICY PREMIUM: $135,000 Gross
COMPANY RETENTION A) $100,000 Each Occurrence/$100,000 Aggregate
Net &/Or Treaty: (Being 10.0000%) Part of $1,000,000 Each
Occurrence/$1,000,000 Aggregate Excess Any/All
- Underlying Insurance and/or Self-Insured
- Retention (SIR)
-
B) $4,000,000 Each Occurrence/$4,000,000 Aggregate
Excess $1,000,000 Each Occurrence/$1,000,000
Aggregate Excess Any/All Underlying Insurance
and/or Self-Insured Retention (SIR)
OTHER FACULTATIVE: Placed Elsewhere by Company
None
REINSURANCE LIMIT(S) A) BASIS OF ACCEPTANCE: Contri

In [32]:
def multiclass_count(text:str):
    '''
    Determines how many classes there are in the text.
    There are 4 classes in total: Gen'l Liab, Auto liab, Excess Auto Liab, and Professional Liab

    input: 
    string that contains the labels extracted from pdf

    outputs: 
    - count of how many unique claases there are (int)
    - names of the unique classes (list of strings)
    '''
    # Regular expression to find patterns that look like the labels
    # We assume labels consist of one or two words ending with 'Liab' 
    # -> if there are new classes, change the following line of code
    pattern = re.compile(r"(Gen'l Liab|Auto Liab|Excess Auto Liab|Professional Liab)")

    # Find all matches in the text
    matches = pattern.findall(text)
    
    # Convert list of matches to a set to remove duplicates
    unique_labels = set(matches)
    
    # Return the count of unique labels and the list of unique labels
    return len(unique_labels), list(unique_labels)

    


In [12]:
# Find the index positions of 'LOSSES' and 'ADDITIONAL INFORMATION'
losses_index = fernlea.find('LOSSES')
additional_info_index = fernlea.find('ADDITIONAL INFORMATION')

# Find the index position of 'LARGE LOSSES'
large_losses_index = fernlea.find('LARGE LOSSES')

# Slice the string to get the portions after 'LOSSES' and 'ADDITIONAL INFORMATION'
losses = fernlea[losses_index + len('LOSSES'):large_losses_index] if losses_index != -1 else ''

# Slice the string to get the parts after 'LARGE LOSSES'
large_losses = fernlea[large_losses_index + len('LARGE LOSSES'):additional_info_index] if large_losses_index != -1 else ''


print("LOSSES:\n", losses)
print("\nLARGE LOSSES:\n", large_losses)

LOSSES:
 : Valuation Date: 05/07/2024
Gen'l Liab Auto Liab Excess Auto Liab
TOTAL TOTAL TOTAL
YEAR INCURRED NUMBER INCURRED NUMBER INCURRED NUMBER
06/01/2023-06/01/2024 $5 2 0 $0 0
06/01/2022-06/01/2023 $44,574 2 0 $0 0
06/01/2021-06/01/2022 $2,245 4 $21,845 10 $0 0
06/01/2020-06/01/2021 $3,298 7 $1,094,083 4 $1,762,338 1
06/01/2019-06/01/2020 $4,826 4 $2,681 5 $0 0
06/01/2018-06/01/2019 $106,121 7 $26,007 5 $0 0


LARGE LOSSES:
 : Valuation Date: 05/07/2024A
Total Line
DOL Incurred O/C Paid Reserve Type Description
03/29/2021 2,846,123 C 2,846,123 0 UL Insured vehicle backing
up to claimant's
receiving dock doors and
insured backed too far
and struck the claimant's
building. Bricks
shattered and came into
the building and the roll
up door frame is damaged.
U12804.245012.01 UMBRELLA LIABILITY Page 2 of 3A/C: Fernlea Industries, Inc. UMBRELLA LIABILITY
R/I: Upland Capital Group June 4, 2024
LARGE LOSSES (continued)
Total Line
DOL Incurred O/C Paid Reserve Type Description
Over 50% at fa

In [13]:
# remove first 4 rows of losses
# Split the text into lines
lines = losses.strip().split('\n')

# Remove the first four lines
lines = lines[4:]

In [14]:
count, lst = multiclass_count(losses)

In [33]:
print(count,lst)

3 ['Excess Auto Liab', 'Auto Liab', "Gen'l Liab"]


In [16]:
# TODO: Finish implementing ExportDf

def ExportDf(text):
    '''
    Process the text string when there is more than one class.
    There are 4 classes in total: Gen'l Liab, Auto liab, Excess Auto Liab, and Professional Liab
    input: string that contains the text extracted from pdf
    output: standardized pandas dataframe
    '''

    count, lst = multiclass_count(text) 

    # define number of columns
    # the formula is 2 (start date and end date) + count * 2 (rach class has 2 columns, total incurred and total number)

    # Define the number of columns based on the formula
    num_columns = 2 * count + 2

    # Create a DataFrame with the required number of columns
    column_names = []
    for label in lst:
        column_names.extend([f"{label} Total Incurred", f"{label} Total Number"])
    column_names.extend(["Effective Date", "Expiration Date", "Evaluation Date"])

    # Initialize the DataFrame with zero rows, just column definitions
    df = pd.DataFrame(columns=column_names)

    print(df)

    return df
    

    


In [ ]:
# TODO: Finish implementing mtach_round1

def match_round1(num_classes:str ):
    '''
    Goes through all lines
    in the variables "lines" (losses without headers) and attemtps 
    to match each line to the default pattern. 

    inputs:

    outputs:
    
    '''

In [55]:
# First round: Extracting data from the text
data_pattern = re.compile(r"(\d{2}/\d{2}/\d{4}-\d{2}/\d{2}/\d{4}) ([\$\d,]+|\$0|\$\d|\S*) (\d+|\d|0) ([\$\d,]+|\$0|\$\d|0|\S*) (\d+|\d|0) ([\$\d,]+|\$0|\$\d|0|\S*) (\d+|\d|0)")

# if there are rows that are not captured, include them
# assuming that the total incurred could be missing

# find matches
matches = data_pattern.findall(losses)

# check if any expected data is missing
if not matches:
    print("No data captured. Please check the losses string format.")
    
else:
    print("Captured Matches:")
    for match in matches:
        print(match)


Captured Matches:
('06/01/2021-06/01/2022', '$2,245', '4', '$21,845', '10', '$0', '0')
('06/01/2020-06/01/2021', '$3,298', '7', '$1,094,083', '4', '$1,762,338', '1')
('06/01/2019-06/01/2020', '$4,826', '4', '$2,681', '5', '$0', '0')
('06/01/2018-06/01/2019', '$106,121', '7', '$26,007', '5', '$0', '0')


In [56]:
losses

": Valuation Date: 05/07/2024\nGen'l Liab Auto Liab Excess Auto Liab\nTOTAL TOTAL TOTAL\nYEAR INCURRED NUMBER INCURRED NUMBER INCURRED NUMBER\n06/01/2023-06/01/2024 $5 2 0 $0 0\n06/01/2022-06/01/2023 $44,574 2 0 $0 0\n06/01/2021-06/01/2022 $2,245 4 $21,845 10 $0 0\n06/01/2020-06/01/2021 $3,298 7 $1,094,083 4 $1,762,338 1\n06/01/2019-06/01/2020 $4,826 4 $2,681 5 $0 0\n06/01/2018-06/01/2019 $106,121 7 $26,007 5 $0 0\n"

In [57]:
print(lines)

['06/01/2023-06/01/2024 $5 2 0 $0 0', '06/01/2022-06/01/2023 $44,574 2 0 $0 0', '06/01/2021-06/01/2022 $2,245 4 $21,845 10 $0 0', '06/01/2020-06/01/2021 $3,298 7 $1,094,083 4 $1,762,338 1', '06/01/2019-06/01/2020 $4,826 4 $2,681 5 $0 0', '06/01/2018-06/01/2019 $106,121 7 $26,007 5 $0 0']


In [68]:
def reformat(line, pattern:re.Pattern, matches:list, pattern_ind:int, line_ind:int):
    '''
    This function takes in a unmatched line in round 1 (due to missing values)
    and reformats it so that
    (1) It's in the standarized format
    (2) missing value defaults to 0
    (3) Adds the reformatted line to the list named "matches"

    inputs: 
    - line (str): a line from the list "unmatched". Lines contains all lines in the loss run table 
    - pattern (re.Pattern): the pattern that the unmatched line matched to in the 2nd round
    - matches (list): the list that contains all lines that matched with the default pattern from the 1st round. Example:
        ('06/01/2021-06/01/2022', '$2,245', '4', '$21,845', '10', '$0', '0')
        ('06/01/2020-06/01/2021', '$3,298', '7', '$1,094,083', '4', '$1,762,338', '1')
        ('06/01/2019-06/01/2020', '$4,826', '4', '$2,681', '5', '$0', '0')
        ('06/01/2018-06/01/2019', '$106,121', '7', '$26,007', '5', '$0', '0')`
    - pattern_ind (int): the index of the pattern

    output:
    - matches (list): OG list with reformatted matches appended
    '''

    match = pattern.search(line)
    print(f"match is {match}")
    
    # Extract the groups from the match
    groups = list(match.groups())

    print(f"Current unmatched line number is {line_ind}.")
    print(f"Current unmatched line is {line}.")

    # Extract the groups from the match, check is matach is none
    groups = list(match.groups())    
    
    # Replace empty values with '0'
    # since we know the 2nd incurred is missing, 
    # add $0 in the position where the second total incurred value should be
    standardized_groups = groups.copy()

    # two ways to do this
    # (1) insert  without checking (faster)
    # check that the index 3 is 0 
    # insert "$0" at index 3
    if (standardized_groups[pattern_ind + 1] == '0'):
        standardized_groups.insert(pattern_ind+1, "$0")
    else: 
        print("The 2nd number of claims is not 0!")

    # (2) check for 0 as claims value (we are not using this yet, wait til testing)
    # for index, g in enumerate(groups):
    #     if index == 3 and g.strip() == '0':  # Check if the position corresponds to the second incurred
    #         standardized_groups.append('$0')  # Insert '$0' for the missing second incurred value
    #     else:
    #         standardized_groups.append(g if g and g.strip() != '$0' else '0')  # Handle other values as usual
            
    # Insert the standardized tuple to the right position in matches
    # use the index, line[0]
    matches.insert(line_ind-1, tuple(standardized_groups))

    return matches


In [69]:
unmatched

[(1, '06/01/2023-06/01/2024 $5 2 0 $0 0'),
 (2, '06/01/2022-06/01/2023 $44,574 2 0 $0 0')]

In [70]:
# 2nd Round: check if there are rows that might not be captured by analyzing the text more closely

# remove first 4 rows of losses
# Split the text into lines
lines = losses.strip().split('\n')

# Remove the first four lines
lines = lines[4:]

expected_rows = len(lines) 

if len(matches) != expected_rows:
    print(f"Expected to capture {expected_rows} rows, but only captured {len(matches)}.")
    missing_rows_count = expected_rows - len(matches)
    print(f"Missing rows count: {missing_rows_count}")

    # track lines that weren't captured
    unmatched = []

    # Analyze each line, add all unmatched lines in the 1st round
    for index, line in enumerate(lines):
        if not data_pattern.search(line):
            unmatched.append((index, line))  # Add 1 to index because line numbers usually start at 1

    print(unmatched)

    # Try alternative patterns to capture these lines, ADD DIFFERENT PATTERNS IF NEEDED
    for line_number, line in unmatched:
        # Pattern with the first incurred loss possibly missing
        pattern1 = re.compile(r"(\d{2}/\d{2}/\d{4}-\d{2}/\d{2}/\d{4}) (\d+|0) ([\$\d,]+|\$0|\$\d|0|\S*) (\d+|0) ([\$\d,]+|\$0|\$\d|0|\S*) (\d+|0)")

        # Pattern with the second incurred loss possibly missing
        pattern2 = re.compile(r"(\d{2}/\d{2}/\d{4}-\d{2}/\d{2}/\d{4}) ([\$\d,]+|\$0|\$\d|\S*) (\d+|0) (\d+|0) ([\$\d,]+|\$0|\$\d|0|\S*) (\d+|0)")
            
        # Pattern with the third incurred loss possibly missing
        pattern3 = re.compile(r"(\d{2}/\d{2}/\d{4}-\d{2}/\d{2}/\d{4}) ([\$\d,]+|\$0|\$\d|\S*) (\d+|0) ([\$\d,]+|\$0|\$\d|0|\S*) (\d+|0) (\d+|0)")

        if pattern1.search(line):
            print(f"Line {line_number} matches pattern1.")

            # call reformat to reformat this line
            reformat(line, pattern1, matches, 1, line_number)
            
        elif pattern2.search(line):
            print(f"Line {line_number} matches pattern2.")

            reformat(line, pattern2, matches, 2, line_number)

        elif pattern3.search(line):
            print(f"Line {line_number} matches pattern3.")

            reformat(line, pattern3, matches, 3, line_number)
        else:
            print(f"Line {line_number} does not match any known pattern.")
    else:
        print("All lines are successfully captured after second check.")
    
else:
    print("All rows are successfully captured after second check.")





All rows are successfully captured after second check.


In [76]:
matches

# organize by start date?

[('06/01/2021-06/01/2022', '$2,245', '4', '$21,845', '10', '$0', '0'),
 ('06/01/2023-06/01/2024', '$5', '2', '$0', '0', '$0', '0'),
 ('06/01/2022-06/01/2023', '$44,574', '2', '$0', '0', '$0', '0'),
 ('06/01/2020-06/01/2021',
  '$3,298',
  '7',
  '$1,094,083',
  '4',
  '$1,762,338',
  '1'),
 ('06/01/2019-06/01/2020', '$4,826', '4', '$2,681', '5', '$0', '0'),
 ('06/01/2018-06/01/2019', '$106,121', '7', '$26,007', '5', '$0', '0')]

In [78]:
for match in matches:
    print(match)

('06/01/2021-06/01/2022', '$2,245', '4', '$21,845', '10', '$0', '0')
('06/01/2023-06/01/2024', '$5', '2', '$0', '0', '$0', '0')
('06/01/2022-06/01/2023', '$44,574', '2', '$0', '0', '$0', '0')
('06/01/2020-06/01/2021', '$3,298', '7', '$1,094,083', '4', '$1,762,338', '1')
('06/01/2019-06/01/2020', '$4,826', '4', '$2,681', '5', '$0', '0')
('06/01/2018-06/01/2019', '$106,121', '7', '$26,007', '5', '$0', '0')


In [80]:
# List to hold each row's data (as dictionaries)
rows = []

# Process each match to create a dictionary for each row
for match in matches:
    date_range, gl_incurred, gl_number, al_incurred, al_number, eal_incurred, eal_number = match
    start_date, end_date = date_range.split('-')
    row = {
        'Gen\'l Liab Total Incurred': gl_incurred,
        'Gen\'l Liab Number': gl_number,
        'Auto Liab Total Incurred': al_incurred,
        'Auto Liab Number': al_number,
        'Excess Auto Liab Total Incurred': eal_incurred,
        'Excess Auto Liab Number': eal_number,
        'Effective Date': start_date,
        'Expiration Date': end_date,
        'Evaluation Date': '05/07/2024'  # TODO: This should be taken from the text
    }
    print(row)
    rows.append(row)

# Define the column names based on the labels found
column_names = [
    'Gen\'l Liab Total Incurred', 'Gen\'l Liab Number',
    'Auto Liab Total Incurred', 'Auto Liab Number',
    'Excess Auto Liab Total Incurred', 'Excess Auto Liab Number',
    'Effective Date', 'Expiration Date', 'Evaluation Date'
]

# Initialize the DataFrame with the row data
df = pd.DataFrame(rows, columns=column_names)

print(df)

{"Gen'l Liab Total Incurred": '$2,245', "Gen'l Liab Number": '4', 'Auto Liab Total Incurred': '$21,845', 'Auto Liab Number': '10', 'Excess Auto Liab Total Incurred': '$0', 'Excess Auto Liab Number': '0', 'Effective Date': '06/01/2021', 'Expiration Date': '06/01/2022', 'Evaluation Date': '05/07/2024'}
{"Gen'l Liab Total Incurred": '$5', "Gen'l Liab Number": '2', 'Auto Liab Total Incurred': '$0', 'Auto Liab Number': '0', 'Excess Auto Liab Total Incurred': '$0', 'Excess Auto Liab Number': '0', 'Effective Date': '06/01/2023', 'Expiration Date': '06/01/2024', 'Evaluation Date': '05/07/2024'}
{"Gen'l Liab Total Incurred": '$44,574', "Gen'l Liab Number": '2', 'Auto Liab Total Incurred': '$0', 'Auto Liab Number': '0', 'Excess Auto Liab Total Incurred': '$0', 'Excess Auto Liab Number': '0', 'Effective Date': '06/01/2022', 'Expiration Date': '06/01/2023', 'Evaluation Date': '05/07/2024'}
{"Gen'l Liab Total Incurred": '$3,298', "Gen'l Liab Number": '7', 'Auto Liab Total Incurred': '$1,094,083', '